In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
%matplotlib inline 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV , LassoCV, Ridge , Lasso
import scipy.stats as st

In [ ]:
df = pd.read_csv('../input/bike-sharing-demand/train.csv')
df1 = pd.read_csv('../input/bike-sharing-demand/test.csv')
sample_submission_df = pd.read_csv('../input/bike-sharing-demand/sampleSubmission.csv')
print(df.shape)
df

In [ ]:
df1.head()

In [ ]:
df.dtypes

In [ ]:
df['datetime'].value_counts()

In [ ]:
df['dayofweek'] = pd.DatetimeIndex(df['datetime']).dayofweek
df.tail(2)

In [ ]:
df['month'] = pd.DatetimeIndex(df['datetime']).month
df.tail(2)

In [ ]:
df = df.drop(['datetime','casual','registered'],axis = 1)

In [ ]:
sns.distplot((df['count']**2))
plt.show()

In [ ]:
sns.distplot(df['count'])
plt.show()

In [ ]:
(np.log(df['count']**2)).isnull().sum()

In [ ]:
#sns.pairplot(df)

In [ ]:
df.corr()

In [ ]:
df['season'].value_counts()

In [ ]:
c = df.groupby(by='season')
c1 = c.get_group(4)
c2 = c.get_group(3)
c3 = c.get_group(2)
c4 = c.get_group(1)

In [ ]:
from scipy.stats import f_oneway, ttest_ind
f_oneway(c1['count'],c2['count'],c3['count'],c4['count'])

- Reject the null hypothesis : => can use this as Potential Variable !!!

In [ ]:
df.boxplot(column = 'count', by = 'holiday')
plt.show()

In [ ]:
h = df.groupby(by='holiday')
h1 = h.get_group(1)
h0 = h.get_group(0)

In [ ]:
ttest_ind(h1['count'], h0['count'])

- NOT a Potential Variable to fit a linear Model !

In [ ]:
df.boxplot(column = 'count', by = 'workingday')

In [ ]:
w = df.groupby(by='workingday')
w1 = w.get_group(1)
w0 = w.get_group(0)

In [ ]:
ttest_ind(w1['count'], w0['count'])

NOT a Potential Variable to fit a linear Model !

In [ ]:
df.boxplot(column = 'count', by = 'weather')
plt.show()

In [ ]:
wh = df.groupby(by='weather')
wh1 = wh.get_group(4)
wh2 = wh.get_group(3)
wh3 = wh.get_group(2)
wh4 = wh.get_group(1)

In [ ]:
f_oneway(wh1['count'],wh2['count'],wh3['count'],wh4['count'])

- It is a potential variable

In [ ]:
df.columns

In [ ]:
df = df.drop(['holiday','workingday', 'month'], axis = 1)

In [ ]:
df.columns

In [ ]:
#sns.pairplot(df)

In [ ]:
df.corr()

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df.drop('count', axis = 1)
y = df['count']
X1 = sm.add_constant(X)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
LR = sm.OLS(y, X1).fit()
LR.summary()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor,BaggingRegressor
import warnings 
from sklearn.exceptions import DataConversionWarning
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

- KNN regressor

In [ ]:
knn = KNeighborsRegressor()
knn_params = {'n_neighbors': np.arange(3,20), 'weights': ['uniform','distance']}
GS = GridSearchCV(knn, knn_params,cv = 5, scoring = 'neg_mean_squared_error' )
GS.fit(X, y)
GS.best_params_

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)
GS.fit(X_scaled, y)
GS.best_params_

- DT Regressor

In [ ]:
dt = DecisionTreeRegressor(random_state = 0)
dt_params = {'max_depth': np.arange(3,55), 'min_samples_leaf' : np.arange(2,20) }
GS_dt = GridSearchCV(dt, dt_params,cv = 10, scoring = 'neg_mean_squared_error' )
GS_dt.fit(X_scaled, y)
GS_dt.best_params_

In [ ]:
KNN = KNeighborsRegressor(n_neighbors = 19, weights = 'uniform')
DT = DecisionTreeRegressor(max_depth = 6, min_samples_leaf = 11, random_state = 0)
RF = RandomForestRegressor(n_estimators = 27, random_state = 0)
AB_RF = AdaBoostRegressor(base_estimator = RF, n_estimators = 100, random_state = 0)
GBoost = GradientBoostingRegressor(n_estimators = 100) 

In [ ]:
models1 = []
models1.append(('KNNRegressor', KNN ))
models1.append(('DTRegressor', DT))
models1.append(('RFRegressor', RF))
models1.append(('ADABoostRegressor',AB_RF))
models1.append(('GradientBoostRegressor',GBoost))


In [ ]:
from sklearn import model_selection
results = []
names = []
for name,model in models1:
    kfold = model_selection.KFold(shuffle = True, n_splits = 10,random_state = 0)
    cv_results = model_selection.cross_val_score(model, X_scaled, y, cv = kfold, scoring = 'neg_mean_squared_log_error')
    #print(cv_results)
    results.append(np.sqrt(np.abs(cv_results))) # every fold, RMSE scores... only for plotting purposes
    names.append(name)
    print("%s:%f(%f)" %(name, np.mean(np.sqrt(np.abs(cv_results))), 
                        np.std(np.sqrt(np.abs(cv_results)), ddof = 1)))

In [ ]:
#from sklearn.ensemble import VotingRegressor
#stacked = VotingRegressor(estimators = [('GradientBoostRegressor',GBoost), ('KNNRegressor', KNN ), ('ADABoostRegressor',AB_RF)])

In [ ]:
models = []
#models.append(('?VotingRegressor', stacked))

In [ ]:
for name,model in models:
    kfold = model_selection.KFold(shuffle = True, n_splits = 17,random_state = 0)
    cv_results = model_selection.cross_val_score(model, X_scaled, y, cv = kfold, scoring = 'neg_mean_squared_error')
    #print(cv_results)
    results.append(np.sqrt(np.abs(cv_results))) # every fold, RMSE scores... only for plotting purposes
    names.append(name)
    print("%s:%f(%f)" %(name, np.mean(np.sqrt(np.abs(cv_results))), 
                        np.std(np.sqrt(np.abs(cv_results)), ddof = 1)))